# 3. Autoencoding Language Models
自动编码语言模型

##### Tokenization

## Preparation for Google Colab

In [ ]:
import os
from google.colab import drive

#  挂载 google 云盘
drive.mount("/content/drive")

print(os.getcwd())  # /content

# print(os.listdir("/content/drive/MyDrive/"))

# print(os.listdir("/content/drive/MyDrive/Colab Notebooks"))

# if os.getcwd() != "/content/drive/MyDrive":
#     os.chdir("/content/drive/MyDrive")

# print(os.getcwd())

In [ ]:
# 提前将 requirements.txt 放在 google 云盘上
!pip install -r /content/drive/MyDrive/requirements.txt

In [3]:
subdir = "ch03c"
work_path = "/content/drive/MyDrive/" + subdir
if not os.path.exists(work_path):
    os.mkdir(work_path)
os.chdir(work_path)
print(os.getcwd())

/content/drive/MyDrive/ch03c


In [4]:
!apt-get install tree && tree -a "./"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (340 kB/s)
Selecting previously unselected package tree.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
./

0 directories, 0 files


## Loading a Turkish Pre-trained Tokenizer

In [5]:
from transformers import AutoTokenizer

tokenizerTUR = AutoTokenizer.from_pretrained(
    "dbmdz/bert-base-turkish-uncased",
)
print(f"VOC size is: {tokenizerTUR.vocab_size}")
print(f"The model is {type(tokenizerTUR)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

VOC size is: 32000
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


## Loading an English Pre-trained Tokenizer

In [6]:
from transformers import AutoTokenizer

tokenizerEN = AutoTokenizer.from_pretrained("bert-base-uncased")
print(f"VOC size is: {tokenizerEN.vocab_size}")
print(f"The model is {type(tokenizerEN)}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

VOC size is: 30522
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [7]:
word_en = "telecommunications"
print(f"is in Turkish Model ? {word_en in tokenizerTUR.vocab}")
print(f"is in English Model ? {word_en in tokenizerEN.vocab}")

is in Turkish Model ? False
is in English Model ? True


In [8]:
tokens = tokenizerTUR.tokenize(word_en)
tokens

['tel', '##eco', '##mm', '##un', '##ica', '##tions']

In [9]:
[t in tokenizerTUR.vocab for t in tokens]

[True, True, True, True, True, True]

In [10]:
tokens = tokenizerEN.tokenize(word_en)
tokens

['telecommunications']

In [11]:
long_word_tur = "Muvaffakiyetsizleştiricileştiriveremeyebileceklerimizdenmişsinizcesine"

'''
It means that “As though you happen to have been from among those whom we will not be able to easily/quickly make a maker of unsuccessful ones”
'''

## Understanding Tokenization Algorithms

### Train tokenizers from scratch

let's load Shakespeare plays from gutenberg project

In [12]:
import nltk
from nltk.corpus import gutenberg

nltk.download("gutenberg")
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
!tree -a /root/nltk_data

/root/nltk_data
├── corpora
│   ├── gutenberg
│   │   ├── austen-emma.txt
│   │   ├── austen-persuasion.txt
│   │   ├── austen-sense.txt
│   │   ├── bible-kjv.txt
│   │   ├── blake-poems.txt
│   │   ├── bryant-stories.txt
│   │   ├── burgess-busterbrown.txt
│   │   ├── carroll-alice.txt
│   │   ├── chesterton-ball.txt
│   │   ├── chesterton-brown.txt
│   │   ├── chesterton-thursday.txt
│   │   ├── edgeworth-parents.txt
│   │   ├── melville-moby_dick.txt
│   │   ├── milton-paradise.txt
│   │   ├── README
│   │   ├── shakespeare-caesar.txt
│   │   ├── shakespeare-hamlet.txt
│   │   ├── shakespeare-macbeth.txt
│   │   └── whitman-leaves.txt
│   └── gutenberg.zip
└── tokenizers
    ├── punkt
    │   ├── czech.pickle
    │   ├── danish.pickle
    │   ├── .DS_Store
    │   ├── dutch.pickle
    │   ├── english.pickle
    │   ├── estonian.pickle
    │   ├── finnish.pickle
    │   ├── french.pickle
    │   ├── german.pickle
    │   ├── greek.pickle
    │   ├── italian.pickle
    │   ├── malayal

In [14]:
!head /root/nltk_data/corpora/gutenberg/shakespeare-macbeth.txt

[The Tragedie of Macbeth by William Shakespeare 1603]


Actus Primus. Scoena Prima.

Thunder and Lightning. Enter three Witches.

  1. When shall we three meet againe?
In Thunder, Lightning, or in Raine?
  2. When the Hurley-burley's done,


In [15]:
import nltk
from nltk.corpus import gutenberg

# /root/nltk_data/corpora/gutenberg/shakespeare-macbeth.txt
# /root/nltk_data/corpora/gutenberg/shakespeare-hamlet.txt
# /root/nltk_data/corpora/gutenberg/shakespeare-caesar.txt
plays = ["shakespeare-macbeth.txt", "shakespeare-hamlet.txt", "shakespeare-caesar.txt"]
# for ply in plays:
#     print("ply:", ply)
#     print(gutenberg.sents(ply))
#     for s in gutenberg.sents(ply):
#         print(" ".join(s))
#     print("--------" * 3)
print(gutenberg.sents("shakespeare-macbeth.txt"))
type(gutenberg.sents("shakespeare-macbeth.txt"))

[['[', 'The', 'Tragedie', 'of', 'Macbeth', 'by', 'William', 'Shakespeare', '1603', ']'], ['Actus', 'Primus', '.'], ...]


nltk.corpus.reader.util.StreamBackedCorpusView

In [16]:
shakespeare = [" ".join(s) for ply in plays for s in gutenberg.sents(ply)]
type(shakespeare)

list

In [17]:
shakespeare[0]

'[ The Tragedie of Macbeth by William Shakespeare 1603 ]'

In [18]:
type(shakespeare[0])

str

In [19]:
shakespeare[0:4]

['[ The Tragedie of Macbeth by William Shakespeare 1603 ]',
 'Actus Primus .',
 'Scoena Prima .',
 'Thunder and Lightning .']

In [20]:
from tokenizers.processors import TemplateProcessing

special_tokens = ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
print("special_tokens:", special_tokens)

temp_proc = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", special_tokens.index("[CLS]")),
        ("[SEP]", special_tokens.index("[SEP]")),
    ],
)

"""
TemplateProcessing(
    single=[
        SpecialToken(id="[CLS]", type_id=0),
        Sequence(id=A, type_id=0),
        SpecialToken(id="[SEP]", type_id=0)
    ],
    pair=[
        SpecialToken(id="[CLS]", type_id=0),
        Sequence(id=A, type_id=0),
        SpecialToken(id="[SEP]", type_id=0),
        Sequence(id=B, type_id=1),
        SpecialToken(id="[SEP]", type_id=1)
    ],
    special_tokens={
        "[CLS]": SpecialToken(id="[CLS]", ids=[1], tokens=["[CLS]"]),
        "[SEP]":SpecialToken(id="[SEP]", ids=[2], tokens=["[SEP]"])
    }
)
"""
print("temp_proc:", temp_proc)

special_tokens: ['[UNK]', '[CLS]', '[SEP]', '[PAD]', '[MASK]']
temp_proc: TemplateProcessing(single=[SpecialToken(id="[CLS]", type_id=0), Sequence(id=A, type_id=0), SpecialToken(id="[SEP]", type_id=0)], pair=[SpecialToken(id="[CLS]", type_id=0), Sequence(id=A, type_id=0), SpecialToken(id="[SEP]", type_id=0), Sequence(id=B, type_id=1), SpecialToken(id="[SEP]", type_id=1)], special_tokens={"[CLS]":SpecialToken(id="[CLS]", ids=[1], tokens=["[CLS]"]), "[SEP]":SpecialToken(id="[SEP]", ids=[2], tokens=["[SEP]"])})


## Training BPE

In [21]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.normalizers import Sequence, NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.decoders import BPEDecoder

# Instantiate BPE (Byte-Pair Encoding)
tokenizer = Tokenizer(BPE())

"""
Tokenizer(
    version="1.0",
    truncation=None,
    padding=None,
    added_tokens=[],
    normalizer=None,
    pre_tokenizer=None,
    post_processor=None,
    decoder=None,
    model=BPE(
        dropout=None,
        unk_token=None,
        continuing_subword_prefix=None,
        end_of_word_suffix=None,
        fuse_unk=False,
        byte_fallback=False,
        ignore_merges=False,
        vocab={},
        merges=[]
    )
)
"""
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[], normalizer=None, pre_tokenizer=None, post_processor=None, decoder=None, model=BPE(dropout=None, unk_token=None, continuing_subword_prefix=None, end_of_word_suffix=None, fuse_unk=False, byte_fallback=False, ignore_merges=False, vocab={}, merges=[]))

In [22]:
# a unicode normalizer, lowercasing and , replacing accents in order  :
# * Sequence : It composes multiple PreTokenizer that will be run in the given order
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])

# Whitespace: Splits on word boundaries using the regular expression \w+|[^\w\s]+
tokenizer.pre_tokenizer = Whitespace()
tokenizer.decoder = BPEDecoder()
tokenizer.post_processor = temp_proc
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[], normalizer=Sequence(normalizers=[NFD(), Lowercase(), StripAccents()]), pre_tokenizer=Whitespace(), post_processor=TemplateProcessing(single=[SpecialToken(id="[CLS]", type_id=0), Sequence(id=A, type_id=0), SpecialToken(id="[SEP]", type_id=0)], pair=[SpecialToken(id="[CLS]", type_id=0), Sequence(id=A, type_id=0), SpecialToken(id="[SEP]", type_id=0), Sequence(id=B, type_id=1), SpecialToken(id="[SEP]", type_id=1)], special_tokens={"[CLS]":SpecialToken(id="[CLS]", ids=[1], tokens=["[CLS]"]), "[SEP]":SpecialToken(id="[SEP]", ids=[2], tokens=["[SEP]"])}), decoder=BPEDecoder(suffix="</w>"), model=BPE(dropout=None, unk_token=None, continuing_subword_prefix=None, end_of_word_suffix=None, fuse_unk=False, byte_fallback=False, ignore_merges=False, vocab={}, merges=[]))

We are ready to train the model

In [23]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(vocab_size=5000, special_tokens=special_tokens)
tokenizer.train_from_iterator(shakespeare, trainer=trainer)
print(f"Trained vocab size: {tokenizer.get_vocab_size()}")

Trained vocab size: 5000


In [24]:
sen = "Is this a dagger which I see before me, the handle toward my hand?"
sen_enc = tokenizer.encode(sen)
print(f"Output: {format(sen_enc.tokens)}")

Output: ['[CLS]', 'is', 'this', 'a', 'dagger', 'which', 'i', 'see', 'before', 'me', ',', 'the', 'hand', 'le', 'toward', 'my', 'hand', '?', '[SEP]']


In [25]:
sen_enc2 = tokenizer.encode("Macbeth and Hugging Face")
print(f"Output: {format(sen_enc2.tokens)}")

Output: ['[CLS]', 'macbeth', 'and', 'hu', 'gg', 'ing', 'face', '[SEP]']


In [26]:
two_enc = tokenizer.encode("I like Hugging Face!", "He likes Macbeth!")
print(f"Output: {format(two_enc.tokens)}")

Output: ['[CLS]', 'i', 'like', 'hu', 'gg', 'ing', 'face', '!', '[SEP]', 'he', 'likes', 'macbeth', '!', '[SEP]']


In [27]:
!tree -a ./

./

0 directories, 0 files


In [28]:
tokenizer.model.save(".")

['./vocab.json', './merges.txt']

In [29]:
!tree -a ./

./
├── merges.txt
└── vocab.json

0 directories, 2 files


In [30]:
!wc -l ./merges.txt

4948 ./merges.txt


In [31]:
!head -6 ./merges.txt

#version: 0.2
t h
o u
a n
th e
r e


In [32]:
!head -1000 ./merges.txt | tail -5

ch ance
si g
your s
ti a
po int


In [33]:
# Save and Load Tokenizer
tokenizer.save("MyBPETokenizer.json")

In [34]:
!tree -a ./

./
├── merges.txt
├── MyBPETokenizer.json
└── vocab.json

0 directories, 3 files


In [35]:
tokenizerFromFile = Tokenizer.from_file("MyBPETokenizer.json")
sen_enc3 = tokenizerFromFile.encode("I like HuggingFace and Macbeth")
print(f"Output: {format(sen_enc3.tokens)}")

Output: ['[CLS]', 'i', 'like', 'hu', 'gg', 'ing', 'face', 'and', 'macbeth', '[SEP]']


## Training WordPiece

In [36]:
from tokenizers.models import WordPiece
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers.normalizers import BertNormalizer

# BERT normalizer includes cleaning the text, handling accents, chinese chars and lowercasing
tokenizer = Tokenizer(WordPiece())
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[], normalizer=None, pre_tokenizer=None, post_processor=None, decoder=None, model=WordPiece(unk_token="[UNK]", continuing_subword_prefix="##", max_input_chars_per_word=100, vocab={}))

In [37]:
tokenizer.normalizer = BertNormalizer()
tokenizer.pre_tokenizer = Whitespace()
tokenizer.decoder = WordPieceDecoder()
tokenizer

Tokenizer(version="1.0", truncation=None, padding=None, added_tokens=[], normalizer=BertNormalizer(clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True), pre_tokenizer=Whitespace(), post_processor=None, decoder=WordPiece(prefix="##", cleanup=True), model=WordPiece(unk_token="[UNK]", continuing_subword_prefix="##", max_input_chars_per_word=100, vocab={}))

In [38]:
from tokenizers.trainers import WordPieceTrainer

trainer = WordPieceTrainer(
    vocab_size=5000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

tokenizer.train_from_iterator(shakespeare, trainer=trainer)
output = tokenizer.encode(sen)
print(output.tokens)

['is', 'this', 'a', 'dagger', 'which', 'i', 'see', 'before', 'me', ',', 'the', 'hand', '##le', 'toward', 'my', 'hand', '?']


In [39]:
tokenizer.decode(output.ids)

'is this a dagger which i see before me, the handle toward my hand?'

In [40]:
tokenizer.encode("Kralsın aslansın Macbeth!").tokens

['[UNK]', '[UNK]', 'macbeth', '!']

## Pre-made tokenizers

* CharBPETokenizer: The original BPE
* ByteLevelBPETokenizer: The byte level version of the BPE
* SentencePieceBPETokenizer: A BPE implementation compatible with the one used by SentencePiece
* BertWordPieceTokenizer: The famous Bert tokenizer, using WordPiece

In [41]:
import tokenizers

"""
from tokenizers.processors import TemplateProcessing

from tokenizers import Tokenizer

from tokenizers.models import BPE
from tokenizers.normalizers import Sequence, NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.decoders import BPEDecoder
from tokenizers.trainers import BpeTrainer

from tokenizers import (
    ByteLevelBPETokenizer,
    CharBPETokenizer,
    SentencePieceBPETokenizer,
    BertWordPieceTokenizer,
    SentencePieceUnigramTokenizer,
)
"""

dir(tokenizers)

['AddedToken',
 'BertWordPieceTokenizer',
 'ByteLevelBPETokenizer',
 'CharBPETokenizer',
 'EncodeInput',
 'Encoding',
 'Enum',
 'InputSequence',
 'List',
 'NormalizedString',
 'OffsetReferential',
 'OffsetType',
 'Offsets',
 'PreTokenizedEncodeInput',
 'PreTokenizedInputSequence',
 'PreTokenizedString',
 'Regex',
 'SentencePieceBPETokenizer',
 'SentencePieceUnigramTokenizer',
 'SplitDelimiterBehavior',
 'TextEncodeInput',
 'TextInputSequence',
 'Token',
 'Tokenizer',
 'Tuple',
 'Union',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'decoders',
 'implementations',
 'models',
 'normalizers',
 'pre_tokenizers',
 'processors',
 'tokenizers',
 'trainers']

In [42]:
from tokenizers import (
    ByteLevelBPETokenizer,
    CharBPETokenizer,
    SentencePieceBPETokenizer,
    BertWordPieceTokenizer,
    SentencePieceUnigramTokenizer,
)

In [43]:
tokenizer = SentencePieceBPETokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)

NFKC()
Metaspace(replacement="▁", prepend_scheme=always, split=True)
Metaspace(replacement="▁", prepend_scheme=always, split=True)
None


In [44]:
tokenizer = BertWordPieceTokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)

BertNormalizer(clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True)
BertPreTokenizer()
WordPiece(prefix="##", cleanup=True)
None
